In [19]:
from pandas.io.json import json_normalize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import matplotlib.cm as cmx

In [20]:
clusters = pd.read_json("./files/cluster.json")

ValueError: Expected object or value

In [21]:
def plotSteps(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        iteration = data.iloc[runIdx]["iteration"]
        clusters = data.iloc[runIdx]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters))
        print("iteration: %s, clusters: %s" % (iteration, len(clusters)))
        for cIdx in range(0, len(clusters)):
            rgba_color = cm.gnuplot(norm(cIdx))
            pointsX = [point[0] for point in clusters[cIdx]["points"]]
            pointsY = [point[1] for point in clusters[cIdx]["points"]]
            ax.scatter(pointsX, pointsY, color=rgba_color)
            ax.scatter(clusters[cIdx]["centroid"][0], clusters[cIdx]["centroid"][1], color=rgba_color, marker="+")

        plt.show()

In [22]:
plotSteps(clusters)

NameError: name 'clusters' is not defined

In [23]:
def plot(data):
    for runIdx in range(0, len(data)):
        fig, ax = plt.subplots()
        clusters_step1 = data.iloc[runIdx]["steps"][0]["clusters"]
        clusters_step2 = data.iloc[runIdx]["steps"][1]["clusters"]
        norm = colors.Normalize(vmin=0, vmax=len(clusters_step1))
        for cIdx in range(0, len(clusters_step1)):
            rgba_color = cm.gnuplot(norm(cIdx))
            ax.plot(clusters_step1[cIdx]["syntheticCenter"], "--", color=rgba_color, label="cluster %s" % cIdx)
            ax.plot(clusters_step2[cIdx]["syntheticCenter"], color=rgba_color, label="cluster %s - rescheduled" % cIdx)
            print(clusters_step2[cIdx]["metric"])
        ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)

        ax.set(xlabel='Hours', ylabel='Power (kW)',title='')
        plt.show()

In [24]:
def plot_aggm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. agg m"], "--", color=rgba_color, label="s1. agg m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. agg m"], color=rgba_color, label="s2. agg m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_maxm(summary, file_name=None): 
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. max m"], "--", color=rgba_color, label="s1. max m")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. max m"], color=rgba_color, label="s2. max m")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Max. Aggregate PAR',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig
    
def plot_peak(summary, file_name=None):
    fig, ax = plt.subplots()
    rgba_color = cm.gnuplot(norm(0))
    ax.plot(range(1, len(summary["s1. agg m"]) + 1), summary["s1. peak"], "--", color=rgba_color, label="s1. peak")
    ax.plot(range(1, len(summary["s2. agg m"]) + 1), summary["s2. peak"], color=rgba_color, label="s2. peak")
    lgd = ax.legend(bbox_to_anchor=(1, -0.3), loc=4, borderaxespad=0.)
    ax.set(xlabel='K clusters', ylabel='Peak power (kW)',title='')
    plt.show()
    if file_name is not None:
        fig.savefig(file_name, bbox_extra_artists=(lgd,), bbox_inches='tight')
    return fig

# PAR Aggregate

In [25]:
summaryBatch_global = pd.read_json("./files/summaryBatch.json")
#summaryBatch_global = summaryBatch_global[["k","clusters","s1. agg m","s1. max m","s1. peak","s2. agg m","s2. max m","s2. peak","total m"]]
#batch_global = pd.read_json("./files/batch.json")

k = len(json_normalize(summaryBatch_global.iloc[0]["step"]))

In [26]:
index_list = []
step_list = []

for i in range(0 , len(summaryBatch_global)):
    step = json_normalize(summaryBatch_global.iloc[i]["step"])
    crossfold = json_normalize(summaryBatch_global.iloc[i]["crossfold"])
    for j in range(0, k):
        index_list.append((crossfold["splits"].values[0], float("{0:.2f}".format(crossfold["sampleSize"].values[0])), step["k"].values[j]))
        step_list.append(step.iloc[j])


index = pd.MultiIndex.from_tuples(index_list, names=["split", "sampleSize", "k"])
summaryBatch_global_treated = pd.DataFrame(step_list, index=index)
summaryBatch_global_treated

clusters  k  s1. agg m  s1. max m  \
split sampleSize k                                                      
1     0.6        1                     [113]  1   1.381087   1.381087   
                 2                  [57, 56]  2   1.001771   1.450742   
                 3              [37, 37, 39]  3   1.049068   1.483215   
                 4          [28, 28, 28, 29]  4   1.015177   1.543310   
                 5      [23, 23, 23, 22, 22]  5   1.046066   1.595152   
                 6  [18, 18, 19, 20, 18, 20]  6   1.014670   1.529094   
      0.7        1                     [132]  1   1.369004   1.369004   
                 2                  [66, 66]  2   1.008400   1.388403   
                 3              [44, 44, 44]  3   1.011260   1.434968   
                 4          [33, 33, 33, 33]  4   1.010871   1.447178   
                 5      [26, 27, 27, 26, 26]  5   1.019723   1.485373   
                 6  [22, 22, 22, 22, 22, 22]  6   1.015998   1.532538   
      0.8        1                     [151]  1   1.430176   1.430176   
                 2                  [75, 76]  2   1.000305   1.430600   
                 3              [51, 50, 50]  3   1.000900   1.460055   
                 4          [37, 37, 38, 39]  4   1.014247   1.490767   
                 5      [30, 30, 31, 30, 30]  5   1.038327   1.521564   
                 6  [25, 25, 25, 26, 25, 25]  6   1.033670   1.577348   
      0.9        1                     [170]  1   1.357453   1.357453   
                 2                  [85, 85]  2   1.001074   1.381724   
                 3              [58, 56, 56]  3   1.014611   1.386491   
                 4          [42, 42, 42, 44]  4   1.008931   1.414701   
                 5      [34, 34, 34, 34, 34]  5   1.010901   1.437743   
                 6  [28, 28, 28, 29, 28, 29]  6   1.044671   1.500470   
      1.0        1                     [189]  1   1.371114   1.371114   
                 2                  [95, 94]  2   1.000200   1.371350   
                 3              [63, 63, 63]  3   1.014644   1.395779   
                 4          [47, 47, 47, 48]  4   1.064519   1.475421   
                 5      [37, 37, 37, 38, 40]  5   1.021058   1.472670   
                 6  [31, 31, 31, 31, 31, 34]  6   1.051584   1.481487   

                      s1. peak   total m  
split sampleSize k                        
1     0.6        1  386.778517  1.381087  
                 2  203.156483  1.381087  
                 3  228.332900  1.381087  
                 4  124.446267  1.381087  
                 5  105.367167  1.381087  
                 6   80.292967  1.381087  
      0.7        1  454.623917  1.369004  
                 2  249.367883  1.369004  
                 3  203.604717  1.369004  
                 4  139.163350  1.369004  
                 5  126.703817  1.369004  
                 6  141.064933  1.369004  
      0.8        1  574.109500  1.430176  
                 2  295.001350  1.430176  
                 3  202.867983  1.430176  
                 4  243.858583  1.430176  
                 5  135.716650  1.430176  
                 6  133.402833  1.430176  
      0.9        1  607.573067  1.357453  
                 2  314.625167  1.357453  
                 3  215.214050  1.357453  
                 4  168.857500  1.357453  
                 5  210.368683  1.357453  
                 6  136.814033  1.357453  
      1.0        1  676.160150  1.371114  
                 2  385.780117  1.371114  
                 3  258.705150  1.371114  
                 4  352.525767  1.371114  
                 5  244.149433  1.371114  
                 6  222.809117  1.371114

In [27]:
k2 = 2
sampleSize = 0.6
query = f"k == {k2} and sampleSize == {sampleSize}"
summaryBatch_global_treated.query(query)["s1. agg m"].mean()
summaryBatch_global_treated.query(query)["s1. max m"].mean()
summaryBatch_global_treated.query(query)["s1. peak"].std()

nan

In [28]:
summaryBatch_global

,crossfold,step
0,"{'splits': 1, 'sampleSize': 0.6000000000000001}","[{'k': 1, 's1. peak': 386.7785166666667, 's1. ..."
1,"{'splits': 1, 'sampleSize': 0.7000000000000001}","[{'k': 1, 's1. peak': 454.6239166666666, 's1. ..."
2,"{'splits': 1, 'sampleSize': 0.8}","[{'k': 1, 's1. peak': 574.1094999999998, 's1. ..."
3,"{'splits': 1, 'sampleSize': 0.9}","[{'k': 1, 's1. peak': 607.5730666666665, 's1. ..."
4,"{'splits': 1, 'sampleSize': 1}","[{'k': 1, 's1. peak': 676.1601499999998, 's1. ..."


In [29]:
with open('summaryBatch.tex','w') as tf:
    tf.write(summaryBatch_global.to_latex())

In [59]:
index_list = []
step_list = []

for j in range(1, k+1):
    for ssize in (x/10 for x in range(6, 11)):
        
        index_list.append((ssize, "mean", j))
        query = f"k == {j} and sampleSize == {ssize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].mean(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].mean(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].mean()}
        s = pd.Series(data,index=["s1. agg m","s1. max m","s1. peak"])
        step_list.append(s)
        
        index_list.append((ssize, "stdv", j))
        query = f"k == {k2} and sampleSize == {sampleSize}"
        data = {"s1. agg m" : summaryBatch_global_treated.query(query)["s1. agg m"].std(),
                "s1. max m" : summaryBatch_global_treated.query(query)["s1. max m"].std(), 
                "s1. peak" : summaryBatch_global_treated.query(query)["s1. peak"].std()}
        s = pd.Series(data,index=["s1. agg m","s1. max m","s1. peak"])
        step_list.append(s)
        
index = pd.MultiIndex.from_tuples(index_list, names=["sampleSize", "metric", "k"])
summaryBatch_global_statistics = pd.DataFrame(step_list, index=index)
summaryBatch_global_statistics

s1. agg m  s1. max m    s1. peak
sampleSize metric k                                  
0.6        mean   1   1.381087   1.381087  386.778517
           stdv   1        NaN        NaN         NaN
0.7        mean   1   1.369004   1.369004  454.623917
           stdv   1        NaN        NaN         NaN
0.8        mean   1   1.430176   1.430176  574.109500
           stdv   1        NaN        NaN         NaN
0.9        mean   1   1.357453   1.357453  607.573067
           stdv   1        NaN        NaN         NaN
1.0        mean   1   1.371114   1.371114  676.160150
           stdv   1        NaN        NaN         NaN
0.6        mean   2   1.001771   1.450742  203.156483
           stdv   2        NaN        NaN         NaN
0.7        mean   2   1.008400   1.388403  249.367883
           stdv   2        NaN        NaN         NaN
0.8        mean   2   1.000305   1.430600  295.001350
           stdv   2        NaN        NaN         NaN
0.9        mean   2   1.001074   1.381724  314.625167
           stdv   2        NaN        NaN         NaN
1.0        mean   2   1.000200   1.371350  385.780117
           stdv   2        NaN        NaN         NaN
0.6        mean   3   1.049068   1.483215  228.332900
           stdv   3        NaN        NaN         NaN
0.7        mean   3   1.011260   1.434968  203.604717
           stdv   3        NaN        NaN         NaN
0.8        mean   3   1.000900   1.460055  202.867983
           stdv   3        NaN        NaN         NaN
0.9        mean   3   1.014611   1.386491  215.214050
           stdv   3        NaN        NaN         NaN
1.0        mean   3   1.014644   1.395779  258.705150
           stdv   3        NaN        NaN         NaN
0.6        mean   4   1.015177   1.543310  124.446267
           stdv   4        NaN        NaN         NaN
0.7        mean   4   1.010871   1.447178  139.163350
           stdv   4        NaN        NaN         NaN
0.8        mean   4   1.014247   1.490767  243.858583
           stdv   4        NaN        NaN         NaN
0.9        mean   4   1.008931   1.414701  168.857500
           stdv   4        NaN        NaN         NaN
1.0        mean   4   1.064519   1.475421  352.525767
           stdv   4        NaN        NaN         NaN
0.6        mean   5   1.046066   1.595152  105.367167
           stdv   5        NaN        NaN         NaN
0.7        mean   5   1.019723   1.485373  126.703817
           stdv   5        NaN        NaN         NaN
0.8        mean   5   1.038327   1.521564  135.716650
           stdv   5        NaN        NaN         NaN
0.9        mean   5   1.010901   1.437743  210.368683
           stdv   5        NaN        NaN         NaN
1.0        mean   5   1.021058   1.472670  244.149433
           stdv   5        NaN        NaN         NaN
0.6        mean   6   1.014670   1.529094   80.292967
           stdv   6        NaN        NaN         NaN
0.7        mean   6   1.015998   1.532538  141.064933
           stdv   6        NaN        NaN         NaN
0.8        mean   6   1.033670   1.577348  133.402833
           stdv   6        NaN        NaN         NaN
0.9        mean   6   1.044671   1.500470  136.814033
           stdv   6        NaN        NaN         NaN
1.0        mean   6   1.051584   1.481487  222.809117
           stdv   6        NaN        NaN         NaN

In [146]:
def getMetrics(metric, summaryMetric):

    index_list = [x/10 for x in range(6, 11)]
    cols = [x for x in range(1, k+1)]
    step_list = []

    for ssize in (x/10 for x in range(6, 11)):
        query = f"sampleSize == {ssize} and metric == \"{summaryMetric}\" "
        dataAllK = summaryBatch_global_statistics.query(query)[f"{metric}"]
        dictAllK = {}
        for j in range(1, k+1):
            d = dataAllK.reset_index().query(f"k == {j}")[f"{metric}"]
            dictAllK[f"{j}"] = d.values
        s = pd.Series(dictAllK, name=f"{ssize}")
        step_list.append(s)

    return pd.DataFrame(step_list)
    
getMetrics("s1. agg m", "mean")
getMetrics("s1. agg m", "stdv")

,1,2,3,4,5,6
0.6,[nan],[nan],[nan],[nan],[nan],[nan]
0.7,[nan],[nan],[nan],[nan],[nan],[nan]
0.8,[nan],[nan],[nan],[nan],[nan],[nan]
0.9,[nan],[nan],[nan],[nan],[nan],[nan]
1.0,[nan],[nan],[nan],[nan],[nan],[nan]


In [147]:
#pd.DataFrame(data=step_list,columns=columns)

step_list

[1    [1.38108744166]
 2    [1.00177092586]
 3    [1.04906803357]
 4    [1.01517731329]
 5    [1.04606623047]
 6    [1.01466953732]
 Name: 0.6, dtype: object, 1    [1.36900369136]
 2    [1.00840031581]
 3    [1.01125956707]
 4     [1.0108707937]
 5    [1.01972325857]
 6    [1.01599808389]
 Name: 0.7, dtype: object, 1    [1.43017606371]
 2    [1.00030469783]
 3     [1.0009003622]
 4    [1.01424653259]
 5    [1.03832656951]
 6    [1.03367014181]
 Name: 0.8, dtype: object, 1    [1.35745282845]
 2    [1.00107422205]
 3    [1.01461060759]
 4    [1.00893097393]
 5    [1.01090096458]
 6    [1.04467149793]
 Name: 0.9, dtype: object, 1    [1.37111387992]
 2    [1.00020042409]
 3    [1.01464386402]
 4    [1.06451873173]
 5    [1.02105799942]
 6    [1.05158365661]
 Name: 1.0, dtype: object]

In [110]:
series = [pd.Series(np.random.rand(3), name=c) for c in list('abcdefg')]
series
pd.DataFrame.from_items([(s.name, s) for s in series])

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  This is separate from the ipykernel package so we can avoid doing imports until


,a,b,c,d,e,f,g
0,0.258748,0.254132,0.370120,0.838616,0.810530,0.776299,0.222378
1,0.925211,0.282097,0.244937,0.973966,0.842747,0.369242,0.273253
2,0.588322,0.918785,0.167355,0.294517,0.887057,0.125352,0.770970
